# Trabalho Prático 1
## Vitor Hugo Lacerda Lana - 2018072301

## Bibliotecas

In [1]:
import sqlite3 as sl
import pandas as pd
pd.set_option('display.max_rows', None)
print("Importação realizada com sucesso")

Importação realizada com sucesso


## Conexão com banco de dados

In [2]:
con = sl.connect('despesas.db')
cur = con.cursor()
print("Conexão Realizada com sucesso")

Conexão Realizada com sucesso


## Consultas SQL Executadas - 1 / 2 / 3 / 4 / 5 / 8
### Consulta  - 1 

In [3]:
Q1 = """
SELECT COD_SUBFUNCAO,
       NOME_SUBFUNCAO
FROM
  (SELECT SF.COD_FUNCAO,
          MAX(SF.CODS_SUB) AS COD_SUB
   FROM
     (SELECT COD_FUNCAO,
             COUNT(COD_SUBFUNCAO) AS CODS_SUB
      FROM SUBFUNCAO
      NATURAL JOIN DETALHAMENTO_FUNCAO
      NATURAL JOIN FUNCAO
      GROUP BY COD_FUNCAO
      ORDER BY COUNT(COD_SUBFUNCAO) DESC) SF)
NATURAL JOIN DETALHAMENTO_FUNCAO
NATURAL JOIN SUBFUNCAO; """

dfq1 = pd.read_sql_query(Q1,con)
dfq1

,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,125,Normatização e fiscalização
1,126,Tecnologia da informação
2,131,Comunicação social
3,244,Assistência comunitária
4,306,Alimentação e nutrição
5,365,Educação infantil
6,511,Saneamento básico rural


### Consulta  - 2

In [4]:
Q2 = """
SELECT *
FROM
  (SELECT 
  SUM(VALOR_DESPESA) TOTAL_DESPESA,
  NOME_FUNCAO,
          NOME_SUBFUNCAO
   FROM FUNCAO
   NATURAL JOIN DETALHAMENTO_FUNCAO
   NATURAL JOIN SUBFUNCAO
   NATURAL JOIN DESPESA_PUBLICA
   WHERE VALOR_DESPESA >0
   GROUP BY NOME_SUBFUNCAO
   ORDER BY NOME_FUNCAO) AS DPS
WHERE NOME_SUBFUNCAO IN
    (SELECT NOME_SUBFUNCAO
     FROM FUNCAO
     NATURAL JOIN DETALHAMENTO_FUNCAO
     NATURAL JOIN SUBFUNCAO
     NATURAL JOIN DESPESA_PUBLICA
     GROUP BY NOME_SUBFUNCAO
     HAVING SUM(VALOR_DESPESA)>200000);
"""
dfq2 = pd.read_sql_query(Q2,con)
dfq2

,TOTAL_DESPESA,NOME_FUNCAO,NOME_SUBFUNCAO
0,227883.77,Assistência social,Assistência comunitária
1,334262.06,Educação,Educação básica


### Consulta 3

In [5]:
Q3 = """
SELECT *
FROM
  (SELECT NOME_ORGAO,
          SUM(VALOR_DESPESA) VALOR_TOTAL
   FROM ORGAO
   NATURAL JOIN DESPESA_PUBLICA
   WHERE ANO_DESPESA = 2018
   GROUP BY NOME_ORGAO)AS OD
ORDER BY OD.VALOR_TOTAL DESC;
"""
dfq3 = pd.read_sql_query(Q3,con)
dfq3

,NOME_ORGAO,VALOR_TOTAL
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,"Ministério da Ciência, Tecnologia, Inovações",0.00
6,Ministério da Fazenda,0.00
7,Ministério da Integração Nacional,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


### Consulta 4

In [6]:
Q4="""
SELECT *
FROM
  (SELECT MES_DESPESA,
          ANO_DESPESA,
          MODALIDADE_DESPESA,
          NOME_PROGRAMA_GOVERNO,
          VALOR_DESPESA
   FROM ORGAO
   NATURAL JOIN DESPESA_PUBLICA
   NATURAL JOIN PROGRAMA_GOVERNO
   WHERE COD_ORGAO='25000')
WHERE MES_DESPESA < 7
ORDER BY MES_DESPESA;
"""
dfq4 = pd.read_sql_query(Q4,con)
dfq4

,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,1,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
1,2,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
2,3,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
3,4,2018,Reserva de Contingência,GESTAO AEROPORTUARIA,0
4,4,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
5,5,2018,Reserva de Contingência,BRASIL SEM MISERIA,0
6,6,2018,Reserva de Contingência,BRASIL SEM MISERIA,0


### Consulta 5

In [7]:
Q5="""
SELECT COD_PROGRAMA_GOVERNO,
       NOME_PROGRAMA_GOVERNO
FROM
  (SELECT COD_PROGRAMA_GOVERNO,
          NOME_PROGRAMA_GOVERNO,
          SUM(VALOR_DESPESA) VALOR_TOTAL
   FROM FUNCAO
   NATURAL JOIN DETALHAMENTO_FUNCAO
   NATURAL JOIN DESPESA_PUBLICA
   NATURAL JOIN PROGRAMA_GOVERNO
   WHERE COD_FUNCAO='12'
     AND ANO_DESPESA='2018'
   GROUP BY NOME_PROGRAMA_GOVERNO) GE
ORDER BY VALOR_TOTAL DESC
LIMIT 0,3;

"""
dfq5=pd.read_sql_query(Q5,con)
dfq5

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,35,MAIS MEDICOS


### Consulta 8

In [8]:
Q8 = """
SELECT NOME_PROGRAMA_GOVERNO,
       NOME_SUBFUNCAO,
       COUNT(NOME_SUBFUNCAO) AS QNT_ENTRADAS,
       SUM(VALOR_DESPESA) AS VALOR_DESPESA_SUBFUNCAO
FROM SUBFUNCAO
NATURAL JOIN DETALHAMENTO_FUNCAO
NATURAL JOIN DESPESA_PUBLICA
NATURAL JOIN PROGRAMA_GOVERNO
GROUP BY NOME_PROGRAMA_GOVERNO,
         NOME_SUBFUNCAO
ORDER BY NOME_PROGRAMA_GOVERNO;
"""
dfq8 = pd.read_sql_query(Q8,con)
dfq8

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,QNT_ENTRADAS,VALOR_DESPESA_SUBFUNCAO
0,BRASIL SEM MISERIA,Alimentação e nutrição,112,100373.55
1,BRASIL SEM MISERIA,Assistência comunitária,117,227883.77
2,BRASIL SEM MISERIA,Comunicação social,1,0.00
3,BRASIL SEM MISERIA,Difusão do conhecimento científico e tecnológico,13,0.00
4,BRASIL SEM MISERIA,Educação infantil,2,0.00
5,BRASIL SEM MISERIA,Extensão rural,17,0.00
6,BRASIL SEM MISERIA,Fomento ao trabalho,1,0.00
7,BRASIL SEM MISERIA,Outros encargos especiais,12,0.00
8,BRASIL SEM MISERIA,Preservação e conservação ambiental,1,0.00
9,BRASIL SEM MISERIA,Promoção comercial,18,0.00


## Tradução dos comandos de Álgebra Relacional

### Comando 12

In [9]:
C12 = """
SELECT NOME_FUNCAO, NOME_SUBFUNCAO
FROM FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO NATURAL JOIN SUBFUNCAO;
"""
dfc12 = pd.read_sql_query(C12,con)
dfc12

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


### Comando 13

In [10]:
C13 = """
SELECT MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO
FROM PROGRAMA_GOVERNO NATURAL JOIN (SELECT * FROM DESPESA_PUBLICA WHERE MODALIDADE_DESPESA='Reserva de Contingência');
"""
dfc13 = pd.read_sql_query(C13,con)
dfc13

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.00,BRASIL SEM MISERIA
1,Reserva de Contingência,0.00,BRASIL SEM MISERIA
2,Reserva de Contingência,0.00,BRASIL SEM MISERIA
3,Reserva de Contingência,0.00,BRASIL SEM MISERIA
4,Reserva de Contingência,0.00,BRASIL SEM MISERIA
5,Reserva de Contingência,0.00,BRASIL SEM MISERIA
6,Reserva de Contingência,0.00,BRASIL SEM MISERIA
7,Reserva de Contingência,0.00,BRASIL SEM MISERIA
8,Reserva de Contingência,0.00,VIOLENCIA NO TRANSITO
9,Reserva de Contingência,0.00,VIOLENCIA NO TRANSITO


### Comando 14

In [11]:
C14 = """
SELECT VALOR_DESPESA, NOME_ORGAO
FROM DESPESA_PUBLICA NATURAL JOIN (SELECT * FROM ORGAO WHERE NOME_ORGAO='Ministério da Fazenda');
"""
dfc14 = pd.read_sql_query(C14,con)
dfc14

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


### Comando 15

In [12]:
C15 = """
SELECT NOME_PROGRAMA_GOVERNO, VALOR_DESPESA
FROM DESPESA_PUBLICA NATURAL JOIN (SELECT * FROM ORGAO WHERE NOME_ORGAO='Ministério da Saúde') NATURAL JOIN PROGRAMA_GOVERNO;
"""
dfc15 = pd.read_sql_query(C15,con)
dfc15

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.00
1,VIVER SEM LIMITE,0.00
2,VIVER SEM LIMITE,0.00
3,VIVER SEM LIMITE,0.00
4,VIVER SEM LIMITE,0.00
5,VIVER SEM LIMITE,0.00
6,VIVER SEM LIMITE,0.00
7,VIVER SEM LIMITE,0.00
8,VIVER SEM LIMITE,0.00
9,VIVER SEM LIMITE,0.00
